<h1><center>The decomposition of realized volatility</center></h1>

## Section I

I decomposed the realized volatility calculation into several components and comducted exploratary data analsysi of these components in this notebook. Similar to  images modelling, the purpose of the decomposition is to simplify the realized volatility as finding edges or texetures of the image. I would like to gain more insights through this exercise in the following: 
- understand the data items and their impacts on realized volatility 
- extract useful features in the forecast of realized volatility
- find a suitable model in the forecast 
- interpret the final model results

To limite the length of this notebook, I will only discuss the decomposition and explortary data analysis in this notbook and leave its other implications in a seperate notebook. Let $t_i$ denotes the time of random event i occurred within a fixed time period $T$. The event means any changes of order or trade books. There are $N$ events occurred within $T$. The following Equation defines $wp_{t_i}$ as a weighted average of ask $p_{t_i}^a$ and bid $p_{t_i}^b$ price by tis coresponding bid and ask orders. In particular, 

$$
\begin{align}
wp_{t_i}  = \frac{p_{t_i}^bq_{t_i}^a + p_{t_i}^aq_{t_i}^b}{q_{t_i}^a+q_{t_i}^b}
\tag{1}
\end{align} 
$$

This definition of weighted price is from a market maker's point view: when it posts an ask order $q^a$ to sell, it can immediately  seel it at a bid price $p^b$ and this works similarly for its bid order to buy. An averaged price defined from an investor's point view is $wp_{t_i}  = \frac{p_{t_i}^aq_{t_i}^a + p_{t_i}^bq_{t_i}^b}{q_{t_i}^a+q_{t_i}^b}$ because the investor has to pay $p^a$ to buy $q^a$ stocks and only get pay $p^b$ to sell $q^b$ stocks . Since market makers are liquidity providers and investors are liquidity consumers, so they have slightly different weighted prices. The decomposition below works in both cases.  I will focus on the market makers' weighted price.    

Let $\frac{q_{t_i}^a}{q_{t_i}^a+q_{t_i}^b}$ denote as size $Q_{t_i}$ and $\frac{p_{t_i}^a}{p_{t_i}^b}$ as spread $S_{t_i}$  we have the following Equation (1):

$$
\begin{align}
\ln{wp_{t_i}} & = \ln{p_{t_i}^b}+ \ln(Q_{t_i}+S_{t_i}(1-Q_{t_i})) \\
              & \approx  \ln{p_{t_i}^b}+ (Q_{t_i}+S_{t_i}(1-Q_{t_i})-1)  
\tag{2}
\end{align}
$$

I only keep the first order approximation and omit the higher order approximation error $\mathcal{O}((Q_{t_i}+S_{t_i}(1-Q_{t_i})-1)^2)$ in Equation (2). This seems sensible because $S$ is close to 1 and therefore $Q+S(1-Q)$ is close to 1. I will examine when this approximation is justified or not using actual data in next section. As a result of Equation (2), the change of $\ln{wp_{t_i}}$ is approximated by:

$$
\begin{align}
\nabla{\ln{wp_{t_i}}}\approx \nabla{\ln{p_{t_i}^b}} + (1-S_{t_i})\nabla{Q_{t_i}}+ (1-Q_{t_i})\nabla{S_{t_i}}   
\tag{3}
\end{align}
$$

Let's define the realized variance in a fixed time interval $T$ as the following: 

$$
\begin{align}
\sum_{i=1}^N {\nabla{\ln{wp_{t_i}}}}^2 & \approx \sum_{i=1}^N {({\nabla{\ln{p_{t_i}^b}}}^2 + (1-S_{t_i})^2 \nabla{Q_{t_i}^2} + (1-Q_{t_i})^2 \nabla{S_{t_i}^2} )} \\
& + \sum_{i=1}^N 2(\nabla{\ln{p_{t_i}^b}}(1-S_{t_i})\nabla{Q_{t_i}}+ \nabla{\ln{p_{t_i}^b}}(1-Q_{t_i})\nabla{S_{t_i}} + (1-S_{t_i})(1-Q_{t_i})\nabla{Q_{t_i}}\nabla{S_{t_i}})
\tag{4}
\end{align}
$$

The relaized volatility is the squre root of realized variance $\sqrt{\sum_{i=1}^N {\nabla{\ln{wp_{t_i}}}}^2}$. The realized varaince is more amenable to analysis because we can show its seperate additive components while we can not do so easily with realized volatility. Notice that the realized valotility (variance) defined above is more complicated that the usual price volatility defined in literature: the price volatility is defined by $\sum_{i=1}^N {\nabla{\ln{p_{t_i}^b}^2}}$, now we have five more terms to consider.   

We are interested in forecasting the realized varaince at time $0$. Using the law of Iterated Expectionm, we have:

$$
\mathbf{E}_{t_0}\sum_{i=1}^N {\nabla{\ln{wp_{t_i}}}}^2= \mathbf{E}_{t_0}\sum_{i=1}^N \mathbf{E}_{t_{i-1}}{\nabla{\ln{wp_{t_i}}}}^2 
$$

In other words, the expection of sum at time $0$ can be written as the sum of conditional expection at future times.  Therefore we are interested in the contitional second moment of $\nabla{\ln{wp_{t_i}}}$ given the information set $\mathcal{F}_{t_{i-1}}$, which is:

$$
\begin{align}
\mathbf{E}{\nabla{\ln{wp_{t_i}}}}^2|\mathcal{F}_{t_{i-1}} \approx &\underbrace{\mathbf{E}_{t_{i-1}}{\nabla{\ln{p_{t_i}^b}}}^2}_{bid\, variance }+ \underbrace{\mathbf{E}_{t_{i-1}}(1-S_{t_i})^2\nabla{Q_{t_i}^2}|\mathcal{F_{t_{i-1}}}}_{order\,variance} +  \underbrace{\mathbf{E}_{t_{i-1}}(1-Q_{t_i})^2 \nabla{S_{t_i}^2}}_{spread\,variance}  \\ 
                                                  & + \underbrace{2\mathbf{E}_{t_{i-1}}\nabla{\ln{p_{t_i}^b}}(1-S_{t_i})\nabla{Q_{t_i}}+ 2\mathbf{E}_{t_{i-1}}\nabla{\ln{p_{t_i}^b}}(1-Q_{t_i})\nabla{S_{t_i}} + 2\mathbf{E}_{t_{i-1}}(1-S_{t_i})(1-Q_{t_i})\nabla{Q_{t_i}}\nabla{S_{t_i}}}_{cross\,terms}
\tag{5}
\end{align}
$$

The second moment or realized variance is decomposed as the sum of six terms in Equation (5): the variance of bid return, the variance of order, the variance of spread and three cross terms.  Beside the term of bid return variance, the other terms are fairly complicated.  For example, the order variance can be further decomposed as the following to show the involed higher moment terms: 

$$ 
\begin{align}
\mathbf{E}_{t_{i-1}}(1-S_{t_i})^2\nabla{Q_{t_i}^2} & = \mathbf{E}_{t_{i-1}}(1-\nabla{S_{t_{i}}}-S_{t_{i-1}})^2\nabla{Q_{t_i}^2} \\
& = S_{t_{i-1}}^2\mathbf{E}_{t_{i-1}}\nabla{Q_{t_i}^2}+ \mathbf{E}_{t_{i-1}}(1-\nabla{S_{t_i})}^2\nabla{Q_{t_i}^2}-2S_{t_{i-1}}\mathbf{E}_{t_{i-1}}(1-\nabla{S_{t_i}})\nabla{Q_{t_i}^2}
\tag{6}
\end{align}
$$

We cannot ignore the higher order dependence between $\nabla{S_{t_i}}$ and $\nabla{Q_{t_i}}$ when calculating the order variance. We will examine each term's importance in next section. Spread variance is very simliar to order variance. We can conduct a simlar decompotion to the corss terms. For example, the first dirt cross term as be written as : 



$$
\begin{align}
2\mathbf{E}_{t_{i-1}}\nabla{\ln{p_{t_i}^b}}(1-S_{t_i})\nabla{Q_{t_i}}&= 2\mathbf{E}_{t_{i-1}}\nabla{\ln{p_{t_i}^b}}(1-\nabla{S_{t_i}}-S_{t_{i-1}})\nabla{Q_{t_i}} \\
                     &= 2(1-S_{t_{i-1}})\mathbf{E}_{t_{i-1}}\nabla{\ln{p_{t_i}^b}}\nabla{Q_{t_i}} -2 \mathbf{E}_{t_{i-1}}\nabla{\ln{p_{t_i}^b}}\nabla{S_{t_i}}\nabla{Q_{t_i}}
\tag{7}
\end{align}                     
$$


The first two cross terms involve the changes in price, order and spread. They are very interesting because they may display different behaviors when the oder book change are due to trading or not. 

For these equations, we notice that the following high order moments are important in the realized variance calculation: $\nabla{S}^2\nabla{Q}^2$,$\nabla{S}\nabla{Q}^2$, $\nabla{Q}\nabla{S}^2$, $\nabla{\ln{p^b}}\nabla{Q}$,$\nabla{\ln{p^b}}\nabla{S}$ and $\nabla{\ln{p^b}}\nabla{Q}\nabla{S}$. It does not inclue the terms such as $\nabla{\ln{p^b}}^2\nabla{Q}$ or $\nabla{Q}\nabla{S}$. I will examin theese higher moments and their implications to realized volatility in next section.  

In the above analysis, I did not distinguish the change in order book due to trade and no trade.  In addition, the randomness of number of order book changes $N$ and how it interact with other variables $\nabla{ln(p^b)}$ , $\nabla{Q}$ and $\nabla{S}$. Under the assumption of that all these varaibles are i.i.ds , the forecate of realized variance is simplified as     

$$ 
\mathbf{E}_{t_0}\sum_{i=1}^N {\nabla{\ln{wp_{t_i}}}}^2=\bar{N}(\mathbf{E}{\nabla{\ln{p^b}}}^2+\mathbf{E}(1-S)^2\mathbf{E}\nabla{Q^2}+\mathbf{E}(1-Q)^2\mathbf{E}\nabla{S^2})  
\tag{8}
$$

I expect that the i.i.d hypothesis will fail when tested by actual data in next section.  However, the purpose of the exercise is to quantify the limitation of this hypothesis. It is worth of noting that Equation (8) can also serve as a naive forecast of realized volatility.  

In addtion, I did not distinguish the change in order book due to trade or not, they may have different implocations in realized volatily forecast. To investgiate such difference, One possible solution is to label the variables based on whether they are triggered by trade or not such that $N = N_{trade}+ N_{no / trade}$     

$$ 
\mathbf{E}_{t_0}\sum_{i=1}^N {\nabla{\ln{wp_{t_i}}}}^2=\mathbf{E}_{t_0}\sum_{k=1}^{N_{trade}}{\nabla{\ln{wp_{t_k}}}}^2 + \mathbf{E}_{t_0}\sum_{j=1}^{N_{no trade}}{\nabla{\ln{wp_{t_j}}}}^2
\tag{9}
$$

Lastly, I need to be more specific about the meaning of information set $\mathcal{F}_{t_{0}}$, i.e. what information we have before the forecast in this competition. It includes all the order book and trade information from time  $-T$ to 0. This is different from the usual time series analysis assumption which assumes that all the past information untile time 0 are known. The implication is that we rely on more cross section inferences in the forecast because the time series is short (10 minutes). In addition, without knowing the time order of each bucket of book data also makes the validation set impossible being leakage free.            

## Section II: Data analysis


In [ ]:
import pandas as pd
import numpy as np
from glob import glob

import seaborn as sns
from matplotlib import pyplot as plt
sns.set() # Setting seaborn as default style even if use only matplotlib

datapath = '../input/optiver-realized-volatility-prediction/'

allpath=glob(datapath+'/book_train.parquet/*')

TINY = 1e-08

# helper functions

def lag_1(series:pd.Series)->pd.Series:
    return series.shift(1)



def log_return(series:pd.Series)->pd.Series:
    return np.log(series).diff()

def series_diff(series:pd.Series)->pd.Series:
    return series.diff()

def realized_variance(series:pd.Series)->pd.Series:
    return np.sum(series**2)

def count_unique(series):
    return len(np.unique(series))


def check_book_data(file_path:str)->pd.DataFrame:
    
    df = pd.read_parquet(file_path)
    trade_path= file_path.replace("book","trade")
    trade_df = pd.read_parquet(trade_path)
    trade_df= trade_df[['time_id','seconds_in_bucket']]
    trade_df['trade_flag'] = 1 

    df = pd.merge(df,trade_df, how='left', on=['time_id','seconds_in_bucket']) 
    df['trade_flag'].fillna(0,inplace=True)

    df['wap1'] = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1']+TINY)
    df.drop(['bid_price2','ask_price2','bid_size2','ask_size2'],axis=1,inplace=True)
    df['Q'] = df['ask_size1'] / (df['bid_size1'] + df['ask_size1']+TINY)
    df['S'] = df['ask_price1'] / (df['bid_price1'] +TINY)


    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)

    df['bid_return1'] = df.groupby(['time_id'])['bid_price1'].apply(log_return)

    df['Q_diff'] = df.groupby(['time_id'])['Q'].apply(series_diff)
    df['S_diff'] = df.groupby(['time_id'])['S'].apply(series_diff)
    
    df['Q_lag1'] = df.groupby(['time_id'])['Q'].apply(lag_1)
    df['S_lag1'] = df.groupby(['time_id'])['S'].apply(lag_1)
    
    
    
    df['ret_approx'] = df['bid_return1'] + (1-df['S'])*df['Q_diff']+ (1-df['Q'])*df['S_diff'] 
    df['ret_approx2'] = df['bid_return1'] + (1-df['S'])*df['Q_diff']+ (1-df['Q'])*df['S_diff'] - 0.5*((1-df['S'])*df['Q_diff']+ (1-df['Q'])*df['S_diff'])**2 
    df['check']=np.log(df['wap1'])-np.log(df['bid_price1'])-np.log(df['Q']+df['S']*(1-df['Q']))
    df['error']=df['ret_approx']-df['log_return1']
    df['error2']=df['ret_approx2']-df['log_return1']
    

    df['stock_id'] = int(file_path.split('=')[1])
    
    return df 


def select_vars(df:pd.DataFrame)->pd.DataFrame:
    res = df[['error','error2','stock_id','time_id']]
    #res.fillna(0,inplace=True)
    #res.replace(np.inf, 0, inplace=True)
    res['error']=df['error'].astype(np.float16)
    res['error2']=df['error2'].astype(np.float16)
    #res['time_id']=df['time_id'].astype(np.int16)
    res['stock_id']=df['stock_id'].astype(np.int16)

    return res

def check_book_wraper(file_path:str)->pd.DataFrame:
    return select_vars(check_book_data(file_path))



def vol_cal(df:pd.DataFrame)->pd.DataFrame:
    df['log_return1_trade'] = df['log_return1']*df['trade_flag']
    df['bid_return1_trade'] = df['bid_return1']*df['trade_flag']

    df['S_diff_square'] =df['S_diff']**2
    df['one_minus_S_square'] =(1-df['S'])**2
    
    df['Q_diff_square'] =df['Q_diff']**2
    
    df['bid_return1_Q_diff'] =df['bid_return1']*df['Q_diff']
    df['bid_return1_S_diff'] =df['bid_return1']*df['S_diff']

    df['S_diff_square_Q_diff_square'] =(df['Q_diff']**2)*(df['S_diff']**2)
    df['S_diff_Q_diff_square'] =(df['Q_diff']**2)*df['S_diff']

    df['Q_diff_S_diff_square'] =(df['S_diff']**2)*df['Q_diff']
    df['bid_return1_Q_diff_S_diff'] =df['bid_return1']*df['Q_diff']*df['S_diff']

     
    df['size_variance'] =((1-df['S'])**2)*(df['Q_diff']**2)
    df['size_variance_check_a'] =df['Q_diff_square']* df['one_minus_S_square']
    
    df['size_variance_ind1'] =(1-df['S'])**2
    df['size_variance_trade'] =df['size_variance']*df['trade_flag']
    



    df['size_variance_a'] =(df['S_lag1']**2)*(df['Q_diff']**2)
    df['size_variance_b'] =((1-df['S_diff'])**2)*(df['Q_diff']**2)
    df['size_variance_c'] =-2*df['S_lag1']*(1-df['S_diff'])*(df['Q_diff']**2)


    df['spread_variance'] =((1-df['Q'])**2)*(df['S_diff']**2)


    df['spread_variance_ind1'] =(1-df['Q'])**2
    

    df['spread_variance_a'] =(df['Q_lag1']**2)*(df['S_diff']**2)
    df['spread_variance_b'] =((1-df['Q_diff'])**2)*(df['S_diff']**2)
    df['spread_variance_c'] =-2*df['Q_lag1']*(1-df['Q_diff'])*(df['S_diff']**2)

    df['spread_variance_trade'] =df['spread_variance']*df['trade_flag']
    


    df['cross1'] =2*df['bid_return1']*(1-df['S'])*df['Q_diff']
    df['cross1_a'] =2*df['bid_return1']*(1-df['S_lag1'])*df['Q_diff']
    df['cross1_b'] =-2*df['bid_return1']*df['S_diff']*df['Q_diff']

    df['cross1_trade'] =df['cross1'] *df['trade_flag']
    

    
    df['cross2'] =2*df['bid_return1']*(1-df['Q'])*df['S_diff']
    df['cross2_a'] =2*df['bid_return1']*(1-df['Q_lag1'])*df['S_diff']
    df['cross2_b'] =-2*df['bid_return1']*df['Q_diff']*df['S_diff']

    df['cross2_trade'] =df['cross2'] *df['trade_flag']
    


    df['cross3'] =2*(1-df['S'])*df['Q_diff']*(1-df['Q'])*df['S_diff']
    df['cross3_a'] =2*(1-df['S_diff'])*df['Q_diff']*(1-df['Q_diff'])*df['S_diff']
    df['cross3_b'] =2*df['S_lag1']*df['Q_diff']*df['Q_lag1']*df['S_diff']
    df['cross3_c'] =-2*(1-df['S_diff'])*df['Q_diff']*df['Q_lag1']*df['S_diff']
    df['cross3_d'] =-2*(1-df['Q_diff'])*df['Q_diff']*df['S_lag1']*df['S_diff']

    df['cross3_trade'] =df['cross3'] *df['trade_flag']
    
    
     
    feature_dict = {


        'error': [np.mean],
        'error2': [np.mean],
        

        'log_return1': [realized_variance],
        'log_return1_trade': [realized_variance],
        
        'bid_return1':[realized_variance],
        'bid_return1_trade':[realized_variance],
        
        'size_variance':[np.sum],
        'size_variance_check_a':[np.sum],
        'size_variance_ind1':[np.sum],
        'size_variance_trade':[np.sum],
        
        
        'size_variance_a':[np.sum],
        'size_variance_b':[np.sum],
        'size_variance_c':[np.sum],
        
        'spread_variance':[np.sum],
        'spread_variance_ind1':[np.sum],
        'spread_variance_trade':[np.sum],
        

        'spread_variance_a':[np.sum],
        'spread_variance_b':[np.sum],
        'spread_variance_c':[np.sum],
        
        
        'cross1':[np.sum],
        'cross1_trade':[np.sum],
        
        'cross1_a':[np.sum],
        'cross1_b':[np.sum],
        
        'cross2':[np.sum],
        'cross2_trade':[np.sum],
        
        'cross2_a':[np.sum],
        'cross2_b':[np.sum],
        
        'cross3':[np.sum],
        'cross3_trade':[np.sum],
        
        'cross3_a':[np.sum],
        'cross3_b':[np.sum],
        'cross3_c':[np.sum],
        'cross3_d':[np.sum],
        
        
        'S_diff_square':[np.sum],
        'Q_diff_square':[np.sum],
        'bid_return1_Q_diff':[np.sum],
        'bid_return1_S_diff':[np.sum],
        'S_diff_square_Q_diff_square':[np.sum],
        'S_diff_Q_diff_square':[np.sum],
        'Q_diff_S_diff_square':[np.sum],
        'bid_return1_Q_diff_S_diff':[np.sum],
        'seconds_in_bucket':[count_unique],
        'trade_flag':[np.sum]
        
    }
    
        
    df_feature = df.groupby(['time_id']).agg(feature_dict).reset_index()
    df_feature.columns = ['_'.join(col) for col in df_feature.columns]
    df_feature['stock_id']= df['stock_id'].unique()[0]
    
    #stock_id = file_path.split('=')[1]
    #df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature['wap_vol'] =np.sqrt(df_feature['log_return1_realized_variance'])
    df_feature['sum_vol'] =np.sqrt(df_feature['bid_return1_realized_variance']+df_feature['size_variance_sum'] + df_feature['spread_variance_sum'])
    df_feature['sum_vol2'] =np.sqrt(df_feature['sum_vol']**2+df_feature['cross1_sum'] + df_feature['cross2_sum']+ df_feature['cross3_sum'])
    df_feature['sum_vol_ind'] =np.sqrt(df_feature['bid_return1_realized_variance']+ df_feature['size_variance_ind1_sum']* df_feature['Q_diff_square_sum']/df_feature['seconds_in_bucket_count_unique']
                                + df_feature['spread_variance_sum']* df_feature['S_diff_square_sum']/df_feature['seconds_in_bucket_count_unique'])
    

    
    df_feature['size_variance_check'] = df_feature['size_variance_sum']/(df_feature['size_variance_a_sum'] + \
                                        df_feature['size_variance_b_sum']+ df_feature['size_variance_c_sum'])
    
    df_feature['spread_variance_check'] = df_feature['spread_variance_sum']/(df_feature['spread_variance_a_sum'] + \
                                        df_feature['spread_variance_b_sum']+ df_feature['spread_variance_c_sum'])
    
    df_feature['cross1_sum_check'] = df_feature['cross1_sum'] /(df_feature['cross1_a_sum']+ \
                                        df_feature['cross1_b_sum'])
    
    df_feature['cross2_sum_check'] = df_feature['cross2_sum'] /(df_feature['cross2_a_sum']+ \
                                        df_feature['cross2_b_sum'])
    
    df_feature['cross3_sum_check'] = df_feature['cross3_sum'] /(df_feature['cross3_a_sum']+ \
                                        df_feature['cross3_b_sum'] + df_feature['cross3_c_sum']+df_feature['cross3_d_sum'])

    return df_feature


def match_trade_order(trade:pd.DataFrame,order:pd.DataFrame)->pd.DataFrame:
    """
    Args:
        Pandas DataFrame: trade
        Pandas DataFrame: order
        
        Both DataFrame have the same time_id, stock_id order_data['idx']=range(order_data.shape[0])
    Returns: 
        Pandas DataFrame:
    """
    df = pd.merge(trade,order, how='left', on=['time_id','seconds_in_bucket'])
    # we simple take the previous index when trade occured, we may add more edge case checks, 
    df['idx']=df['idx']-1
    df.loc[df['idx']<0,'idx']=0  # in case the first trade is at 0 second, but this sould be checked for each time_id 

    pre_order=order.copy() 
    pre_order.columns = ['pre_'+col for col in pre_order.columns]
    pre_order.rename(columns={"pre_idx":"idx"},inplace=True)
    
    df = pd.merge(df,pre_order, how='left', on=['idx'])
    df.drop(['idx'],axis=1,inplace=True)
       

    
    return df


def match_trade_order(trade:pd.DataFrame,order:pd.DataFrame)->pd.DataFrame:
    """
    Args:
        Pandas DataFrame: trade
        Pandas DataFrame: order
        
        Both DataFrame have the same time_id, stock_id order_data['idx']=range(order_data.shape[0])
    Returns: 
        Pandas DataFrame:
    """
    df = pd.merge(trade,order, how='left', on=['time_id','seconds_in_bucket'])
    # we simple take the previous index when trade occured, we may add more edge case checks, 
    df['idx']=df['idx']-1
    df.loc[df['idx']<0,'idx']=0  # in case the first trade is at 0 second, but this sould be checked for each time_id 

    pre_order=order.copy() 
    pre_order.columns = ['pre_'+col for col in pre_order.columns]
    pre_order.rename(columns={"pre_idx":"idx"},inplace=True)
    
    df = pd.merge(df,pre_order, how='left', on=['idx'])
    df.drop(['idx'],axis=1,inplace=True)
   
    return df
       


       




First, I examine the approximation of Equation (3) using actual data, i.e. the log return of weighted price can be approximated by the sum of $\ln{p_{t_i}^b} $ and terms of $Q_t$ and $S_t$.   

In [ ]:
from joblib import Parallel, delayed

def compare_vol(filepath:str)->pd.DataFrame:
    return vol_cal(check_book_data(filepath))

all_df2 = Parallel(n_jobs = -1, verbose = 1)(delayed(compare_vol)(filepath) for filepath in allpath)
all_df2 = pd.concat(all_df2, ignore_index = True)
all_df2[['error_mean','error2_mean']].describe()

The above table summarize the apprimztion error, which is defined as the difference between the acutal and the aprromation of log return of weighted price. For example, in the column error_mean, the mean of the approximation error is about $6 \times 10^{-8}$. The column of erro2_mean is the arrpoximation with second order derivatives. The second order approxiamtion reduces the the mean of error by about $4 \times 10^{-8}$.  Given that adding the second deriavtive will significantly co,plicate the analysis and the gain in accuracy accuracy is moderate, I will use the frist order approxiamtion in teh following. Even these overall statistics are good, thay may ignore the cross sectioanl and time series variations in the approximation error. The following figure shows the approximation error across stock and time buckets. Enen though the mean of appximation error is 0 for all stocks, there are noticable outlires requirng further investigations.         

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
 
sns.set(rc = {'figure.figsize':(20,10)})

sns.violinplot(x="stock_id", y="error_mean", data=all_df2)


Plotting the data before diving into the realized volatilty decomposition is helpful to gain some intuitions. I randomly choose the book data of stock 97 with time id 5 as an example. In each time id, there are 10 minutes (600 seconds) of book data at the resolution of one seond. The figurs show that the log return of weighted price is more volatile than the bid price return. The clues may be in the order size ratio $Q_t$ and spread ratio $S_t$. In particular, the order size ratio $Q_t$ swings widely from 0, its minumum, to 1, its maxmium either with or without trade. The spread ratio $S_t$ is less volatile than $Q_t$. So the market makers frequently adjust order book size to respond to the market changes. Trades happened much less frequently less than the order book changes. Nevertheless trades seems triggered clusters of order book changes, so it seems possible that the order book changes are different when they are associated with a trade or not. 

In [ ]:
trade_data = pd.read_parquet(datapath+'trade_train.parquet/stock_id=97')
order_data = pd.read_parquet(datapath+'book_train.parquet/stock_id=97')

df = check_book_data(datapath+'book_train.parquet/stock_id=97')
order_data['idx']=range(order_data.shape[0])
new_df = match_trade_order(trade_data,order_data)
new_df['vol_bid_ask_size_ratio']=new_df['size']/(new_df['pre_bid_size1']+new_df['pre_ask_size1'])
new_df['trade_ret']=new_df['price']/(new_df['price'].shift(1))-1

plot_df = pd.DataFrame(range(600),columns=['seconds_in_bucket'])
plot_df=pd.merge(plot_df,df[(df['time_id']==5) & (df['stock_id']==97)],how='left',on=['seconds_in_bucket'])[['seconds_in_bucket','Q','S','bid_return1','log_return1']]
plot_df.ffill(axis=0,inplace=True)
plot_df.head()

plot_df2 = pd.DataFrame(range(600),columns=['seconds_in_bucket'])
plot_df2=pd.merge(plot_df2,new_df[new_df['time_id']==5],how='left',on=['seconds_in_bucket'])[['seconds_in_bucket','trade_ret','vol_bid_ask_size_ratio']]
plot_df2.ffill(axis=0,inplace=True)

In [ ]:


#sns.lineplot(x=plot_df['seconds_in_bucket'].values, y=plot_df['Q'].values)
fig, axes = plt.subplots(3, 2, sharex=True, figsize=(24,12))
#fig.suptitle('2 rows x 2 columns axes with no data')

sns.lineplot(x = 'seconds_in_bucket', y = 'log_return1',data = plot_df, ax = axes[0,0])
axes[0,0].set_title("WAP return")
axes[0,0].set(xlabel="Seconds")

sns.lineplot(x = 'seconds_in_bucket', y = 'bid_return1',data = plot_df, ax = axes[0,1])
axes[0,1].set_title("Bid return")
axes[0,1].set(xlabel="Seconds")

sns.lineplot(x = 'seconds_in_bucket', y = 'Q',data = plot_df, ax = axes[1,0])
axes[1,0].set_title("Order size ratio")
axes[1,0].set(xlabel="Seconds")

sns.lineplot(x = 'seconds_in_bucket', y = 'S',data = plot_df, ax = axes[1,1])
axes[1,1].set_title("Spread ratio")
axes[1,1].set(xlabel="Seconds")


sns.lineplot(x = 'seconds_in_bucket', y = 'trade_ret',data = plot_df2, ax = axes[2,0])
axes[2,0].set_title("Trade return")
axes[2,0].set(xlabel="Seconds")
plot_df = pd.DataFrame(range(600),columns=['seconds_in_bucket'])
plot_df=pd.merge(plot_df,df[(df['time_id']==5) & (df['stock_id']==97)],how='left',on=['seconds_in_bucket'])[['seconds_in_bucket','Q','S','bid_return1','log_return1']]
plot_df.ffill(axis=0,inplace=True)
plot_df.head()

plot_df2 = pd.DataFrame(range(600),columns=['seconds_in_bucket'])
plot_df2=pd.merge(plot_df2,new_df[new_df['time_id']==5],how='left',on=['seconds_in_bucket'])[['seconds_in_bucket','trade_ret','vol_bid_ask_size_ratio']]
plot_df2.ffill(axis=0,inplace=True)
sns.lineplot(x = 'seconds_in_bucket', y = 'vol_bid_ask_size_ratio',data = plot_df2, ax = axes[2,1])
axes[2,1].set_title("Vol bid ask size ratio")
axes[2,1].set(xlabel="Seconds")




#sns.lineplot(ax=axes[0], x=plot_df['seconds_in_bucket'].vaaxes[0].set_title("First")lues, y=plot_df['Q'].values)
# axes[0].set_title('Q ')




The following data analysis will quantify the intuitions gained from the above ploting. The previous analysis shows that the higher moments $\nabla{S}^2$,$\nabla{Q}^2$, $\nabla{\ln{p^b}^2}$, $\nabla{S}^2\nabla{Q}^2$,$\nabla{S}\nabla{Q}^2$, $\nabla{Q}\nabla{S}^2$, $\nabla{\ln{p^b}}\nabla{Q}$,$\nabla{\ln{p^b}}\nabla{S}$ and $\nabla{\ln{p^b}}\nabla{Q}\nabla{S}$ are important in realized volatility calculation.  The following table summarize the statistics of these moments using actual data:      

In [ ]:
all_df2[['S_diff_square_sum',
        'Q_diff_square_sum',
        'bid_return1_realized_variance',
        'S_diff_square_Q_diff_square_sum',
        'S_diff_Q_diff_square_sum',
        'Q_diff_S_diff_square_sum',
        'bid_return1_Q_diff_sum',
        'bid_return1_S_diff_sum',
        'bid_return1_Q_diff_S_diff_sum',
        'seconds_in_bucket_count_unique']].describe()

From the table, we notice that the mean of (the sum of) $\nabla{Q}^2$ is 25, which is about $10^6$ larger than the bid price variance $\nabla{\ln{p^b}^2}$, which is 0.000018. Even though $\nabla{Q}^2$ is adjusted by $(1-S_t)^2$ in volatility calculation, we expect that it palys an important role in realized volatility.  In addition, $Q_t$ has two significant cross terms $\nabla{S}\nabla{Q}^2$ and $\nabla{\ln{p^b}}\nabla{Q}$: whose means are 0.000166 and 0.001384. Bother terms are positive on average and we may link them to market makers' behaviors. For example, $\nabla{\ln{p^b}}\nabla{Q}$ is positive becasue when $\nabla{\ln{p^b}}$ goes down due to a sell order, then bid order increases realtive to the ask order, hence $Q_t$ decreases too. In addition, $\nabla{S}\nabla{Q}^2$  is positive on average because market makers increrase the bid-ask spread associated with a large change in $Q_t$. The negative $\nabla{\ln{p^b}}\nabla{S}$ means that market makers increase bid-ask spread when price goes down and tighten it when price increase.  The rest of high moment terms are relatively small. For example, the $\nabla{\ln{p^b}}\nabla{Q}\nabla{S}$ is in the magnitude of $10^{-10}$. These terms may be not important in realized volatility.            

Now I will examine the decomposition results.  As shown in Equation (4) and (5), the realized volatility can be decomposed as the sum of bid price volatility, size volatility, spread volatility and cross terms. As discueed in previous section, the decomposition can be simplified by additional assumptions. For example, we can assume that all the cross term are 0,  or we can further assume that bid price, size,spread and order book change time are independent. These additional assumptions provided us with different decompositions. I compare the ratio of volatility decomposition and its variants to the actual realized volatility.  If the decomposition is the same as the actial realized volatility, the ratio is 1. The following table shows the summary statistics. The columns vol_ration_ind, vol_ratio and vol_ratio2 are the decomposition assuming independence, 0 cross terms and decomposition without additional assumptions.  The table shows that teh raio is 1.12 on average if we assume the total independence. And the ratio is 1.18 if we assume the cross terms are 0.  The approximation is 1.03 if we use Equation (4) without these assumptions. It is clear that the decompositions systematically overestimate the realized volatility if we assume the cross terms are 0 or full independence.                

In [ ]:
all_df2['vol_ratio']=all_df2['sum_vol']/all_df2['wap_vol']
all_df2['vol_ratio2']=all_df2['sum_vol2']/all_df2['wap_vol']
all_df2['vol_ratio_ind']=all_df2['sum_vol_ind']/all_df2['wap_vol']
all_df2[['vol_ratio_ind','vol_ratio','vol_ratio2']].describe()

In [ ]:
def scatter_line_plot(df, compare_col:str, ytitle:str):
    plt.scatter(df['wap_vol'].values,df[compare_col].values)
    plt.plot(df['wap_vol'].values, df['wap_vol'].values,color='red')
    plt.xlabel('WAP volatility')
    plt.ylabel(ytitle)
    #plt.show()

In [ ]:
f, axs = plt.subplots(1,3)
plt.subplot(1,3,1)
scatter_line_plot(all_df2,'sum_vol_ind','Sum of Independent variance')
plt.subplot(1,3,2)
scatter_line_plot(all_df2,'sum_vol','Sum of Variance')
plt.subplot(1,3,3)
scatter_line_plot(all_df2,'sum_vol2','Sum of Variance with cross terms')
f.set_figheight(8)
f.set_figwidth(15)
#f.subtitle('Compare realized volatility calculations')
plt.show()



In the above fifure, the read line shows that if the decompostion and actual valatility is the same. It shows how the decomposition with additional assumptions (the left and middle figures) overly estimate the actual volatility. The decomposition without additional assumptions performered much better. We need be cautious to draw the conclusion that the decomposition assuming independence seems better than the decompostion assuming 0 cross terms. I think the choice to use the actual number of order book change $N$ may help the indepence decomposition. 

In additiion to the summary information of volatility decomposition, the following figure illustrates the decomposotion results across stocks and time buckets. Alrough the bulk of data are around 1, there are abnormal cases worth of further investigations. The decomposition without simplyfying assummption has the best decomposition results.     

In [ ]:
# box figure 
f, axs = plt.subplots(3,1)

plt.subplot(3,1,1)
sns.violinplot(x="stock_id", y="vol_ratio_ind", data=all_df2)
plt.subplot(3,1,2)
sns.violinplot(x="stock_id", y="vol_ratio", data=all_df2)
plt.subplot(3,1,3)
sns.violinplot(x="stock_id", y="vol_ratio2", data=all_df2)

f.set_figheight(15)
f.set_figwidth(25)

The following table shows the cases when the decompistion volatility is very different from the actual realized volatility. 

In [ ]:
bad_decomposition_cases=pd.concat([all_df2.iloc[np.argsort(all_df2['vol_ratio2'].to_numpy())[:10]],all_df2.iloc[np.argsort(all_df2['vol_ratio2'].to_numpy())[-10:]]])
bad_decomposition_cases[['stock_id','time_id_','vol_ratio2']]


The results that cross terms are important in realized volatilty decomposition also implies that the decompostion terms are not independent. The following bar chart shows the contribution of each components to total realized variance. For example, the bid price variance and size variance contribute more that 60% and 40% of realized variance respectively.     

In [ ]:
all_df2[['log_return1_realized_variance','bid_return1_realized_variance','size_variance_sum','spread_variance_sum','cross1_sum','cross2_sum','cross3_sum']].describe()

In [ ]:
barplot_data=pd.DataFrame()
barplot_data['decomposition']=['bid_variance','size_variance','Spread_variance','Cross1','Cross2','Cross3']
barplot_data['data']=[(all_df2['bid_return1_realized_variance']/all_df2['log_return1_realized_variance']).mean(),
                        (all_df2['size_variance_sum']/all_df2['log_return1_realized_variance']).mean(),
                        (all_df2['spread_variance_sum']/all_df2['log_return1_realized_variance']).mean(),
                        (all_df2['cross1_sum']/all_df2['log_return1_realized_variance']).mean(),
                        (all_df2['cross2_sum']/all_df2['log_return1_realized_variance']).mean(),
                        (all_df2['cross3_sum']/all_df2['log_return1_realized_variance']).mean()]

ax = sns.barplot(x="decomposition", y="data", data=barplot_data)                        

We can understand the decomposition results using the previous higher moments results. For example, the corss term1 $\nabla{\ln{p^b}}(1-S)\nabla{Q}$ is negative because $1-S$ is negative and the the moment of $\nabla{\ln{p^b}}\nabla{Q}$ is positive. The moment $\nabla{\ln{p^b}}\nabla{Q}$ is linked to market makers' actions. Similarly, the cross term2 is realted to the moment of $\nabla{\ln{p^b}}\nabla{S}$ and $1-Q$ is positive. Therefore, the negative contributions of cross term 1 and 2 to realized volatility are from market makers' actions.            

In [ ]:
stock_plot_data=pd.DataFrame()
stock_plot_data['stock_id']=all_df2['stock_id']
stock_plot_data['bid_return1_realized_variance']=all_df2['bid_return1_realized_variance']/all_df2['log_return1_realized_variance']

ax= sns.violinplot(x="stock_id", y="bid_return1_realized_variance", data=stock_plot_data)

The following table shows the correlation matrix of each decomposition components. For example, the correlation between realized volatility and size variance volatility is 0.95. 

In [ ]:
corr = all_df2[['log_return1_realized_variance','bid_return1_realized_variance','size_variance_sum','spread_variance_sum','cross1_sum','cross2_sum','cross3_sum','seconds_in_bucket_count_unique']].corr()
cmap=sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

In [ ]:

# corr2 = all_df2[['log_return1_realized_variance','bid_return1_realized_variance','size_variance_a_sum','size_variance_b_sum','size_variance_c_sum',\
#         'spread_variance_a_sum','spread_variance_b_sum','spread_variance_c_sum','cross1_a_sum','cross1_b_sum',\
#         'cross2_a_sum','cross2_b_sum','cross3_a_sum','cross3_b_sum','cross3_c_sum','cross3_d_sum']].corr()


# plot the heatmap
# sns.heatmap(corr, 
#         xticklabels=corr.columns,
#         yticklabels=corr.columns)

# corr2.style.background_gradient(cmap, axis=1)\
#     .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
#     .set_caption("Hover to magify")\
#     .set_precision(2)\
#     .set_table_styles(magnify())

The folliwng figure shows the contribution form each PCA components. The frist three components capture abouit 90% variance of realazied variance. 

In [ ]:

from sklearn.preprocessing import StandardScaler
pca_data = all_df2[['bid_return1_realized_variance','size_variance_sum','spread_variance_sum','cross1_sum','cross2_sum','cross3_sum','seconds_in_bucket_count_unique']] 
scaled_pca_data = StandardScaler().fit_transform(pca_data)
from sklearn.decomposition import PCA
pca = PCA()
scaled_umap_data_pca =pd.DataFrame(pca.fit_transform(scaled_pca_data))
scaled_umap_data_pca 
pd.DataFrame(pca.explained_variance_ratio_).plot.bar()


The following loading matrix of PCA shows that the first PCA component is the sum of bid return, size and Spread variance substrscting the first two cross terms. The second PCA component still includes the sum of bid return, size and Spread variance, which shows the dependenc among these components are not linear.     

In [ ]:
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
loading_matrix = pd.DataFrame(loadings, columns=['PC'+str(i) for i in range(1,8,1)], index=pca_data.columns)
loading_matrix

It is not surprising that the demposition componets of realized volatility are not independent because all of them depend on the number of changes $N_t$.  It is possible to quantify such effects by various monte carlo simulations. For example,  we can calculate the realized volatility and its decompostion by resampling the order book for each stock and time bucket. The simulation will be time consuming and I will leave it to further investigaton.        

Finally I examine how trades or no trade change the realized volatility decomposition. The following table shows the decomposition of realized variance due to trade and no trade. The results are striking: the bid varince is mainly from trades and they contribute more than 50% of actual realized variance. On the contrary, the size variance is mainly from no trades and they contribute mroe than 30% of actual realized variance.      

In [ ]:
barplot_data=pd.DataFrame()
barplot_data['decomposition']=['bid_variance','bid_variance','size_variance','size_variance','Spread_variance','Spread_variance',
                            'Cross1','Cross1','Cross2','Cross2','Cross3','Cross3']
barplot_data['trade']= ['Yes','No']*6                            
barplot_data['data']=[(all_df2['bid_return1_trade_realized_variance']/all_df2['log_return1_realized_variance']).mean(),
                    ((all_df2['bid_return1_realized_variance']-all_df2['bid_return1_trade_realized_variance'])/all_df2['log_return1_realized_variance']).mean(),
                    (all_df2['size_variance_trade_sum']/all_df2['log_return1_realized_variance']).mean(),
                    ((all_df2['size_variance_sum']-all_df2['size_variance_trade_sum'])/all_df2['log_return1_realized_variance']).mean(),
                    (all_df2['spread_variance_trade_sum']/all_df2['log_return1_realized_variance']).mean(),
                    ((all_df2['spread_variance_sum']-all_df2['spread_variance_trade_sum'])/all_df2['log_return1_realized_variance']).mean(),
                    (all_df2['cross1_trade_sum']/all_df2['log_return1_realized_variance']).mean(),
                    ((all_df2['cross1_sum']-all_df2['cross1_trade_sum'])/all_df2['log_return1_realized_variance']).mean(),
                    (all_df2['cross2_trade_sum']/all_df2['log_return1_realized_variance']).mean(),
                    ((all_df2['cross2_sum']-all_df2['cross2_trade_sum'])/all_df2['log_return1_realized_variance']).mean(),
                    (all_df2['cross3_trade_sum']/all_df2['log_return1_realized_variance']).mean(),
                    ((all_df2['cross3_sum']-all_df2['cross3_trade_sum'])/all_df2['log_return1_realized_variance']).mean()]


ax = sns.barplot(x="decomposition", y="data", hue="trade",data=barplot_data)                        

The following figure shows the ratio of trade component to its corresponding volatility decomposition component. For example. the trade related size variance is about 75% of the size variance.   

In [ ]:
trade_barplot_data=pd.DataFrame()
trade_barplot_data['trade']=['realised_varaince','bid_variance','size_variance','Spread_variance','Cross1','Cross2','Cross3']
trade_barplot_data['data']=[(all_df2['log_return1_trade_realized_variance']/all_df2['log_return1_realized_variance']).mean(),
                        (all_df2['bid_return1_trade_realized_variance']/all_df2['bid_return1_realized_variance']).mean(),
                        (all_df2['size_variance_trade_sum']/all_df2['size_variance_sum']).mean(),
                        (all_df2['spread_variance_trade_sum']/all_df2['spread_variance_sum']).mean(),
                        (all_df2['cross1_trade_sum']/all_df2['cross1_sum']).mean(),
                        (all_df2['cross2_trade_sum']/all_df2['cross2_sum']).mean(),
                        (all_df2['cross3_trade_sum']/all_df2['cross3_sum']).mean()]

ax = sns.barplot(x="trade", y="data", data=trade_barplot_data)                        





The following figure illustrates the ratio of trade realted bid return volatility to the bid return volatility across stocks and time buckets. We can see the significant cross section and time series variations.   

In [ ]:
stock_plot_data=pd.DataFrame()
stock_plot_data['stock_id']=all_df2['stock_id']
stock_plot_data['bid_return1_trade_realized_variance']=all_df2['bid_return1_trade_realized_variance']/all_df2['log_return1_realized_variance']

ax= sns.violinplot(x="stock_id", y="bid_return1_trade_realized_variance", data=stock_plot_data)

In [ ]:

# reducer = umap.UMAP()
# embedding = reducer.fit_transform(scaled_umap_data)
# embedding.shape

#import matplotlib.pyplot as plt
# plt.scatter(
#     embedding[:, 0],
#     embedding[:, 1],cmap='Spectral')

#umap.plot.points(embedding)


In [ ]:
# from sklearn.preprocessing import KBinsDiscretizer
# est = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')
# Xt=est.fit_transform(np.expand_dims(all_df2['log_return1_realized_variance'].values,axis=1))

# np.max(Xt)
# scaled_umap_data_pca['target'] =Xt
# import seaborn as sns
# plt.figure(figsize=(12,8))
# sns.scatterplot(
#     x=0, y=1,
#     hue="target",
#     palette=sns.color_palette("bright",n_colors=3),
#     data=scaled_umap_data_pca,
#     legend="full",
#     alpha=0.3
# )
# from sklearn.manifold import TSNE
# X_embedded = TSNE(n_components=2, learning_rate='auto',init='random').fit_transform(scaled_umap_data)

In [ ]:
# from joblib import Parallel, delayed

# all_df = Parallel(n_jobs = -1, verbose = 1)(delayed(check_book_wraper)(filepath) for filepath in allpath)
# all_df = pd.concat(all_df, ignore_index = True)
# #all_df.plot.scatter(x='log_return1',y='ret_approx')

# all_df[['error','error2']].describe()

# all_df.iloc[from joblib import Parallel, delayed

# def compare_vol(filepath:str)->pd.DataFrame:
#     return vol_cal(check_book_data(filepath))np.nanargmin(all_df['error'].to_numpy())-1:np.nanargmin(all_df['error'].to_numpy())+1]
# all_df.iloc[np.nanargmax(all_df['error'].to_numpy())-1:np.nanargmax(all_df['error'].to_numpy())+1]

# import gc
# del all_df
# gc.collect()

## Conclusions

I studied the components of ten minutes realized volatility using actual order book and trade data at 1 second frequency. Besides the price volatility due to trades, the analysis reveals other important contributions to realized volatility, such as size volatility when there are no trades, and cross terms due to market makers' actions. The analysis also indicates cross section and time series variations, including abnormal cases, are woth of further investigations. I will explore the implications of feature engineering and modelling in next note book.    